In [3]:
#Printing hello world

print("Hello")
print("Helo world!")

Hello
Helo world!


In [4]:
import pandas as pd
import os

# File path
file_path = r"c:\python\Quant Research\prices_hist_1Y_amer_20250301.csv"

# Check if the file exists and load the CSV file
if not os.path.exists(file_path):
    print(f"Error: The file at {file_path} was not found. Please verify the file path.")
else:
    # Load the CSV file into a Pandas DataFrame
    try:
        price_1Y = pd.read_csv(file_path)
        print("File loaded successfully!")
        print(price_1Y.head())  # Display the first few rows of the DataFrame
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")



File loaded successfully!
      TICKER                               NAME           EXCHANGE_DESC  \
0  0QBON-MEX              Coloplast A/S Class B  Mexican Stock Exchange   
1  0R87N-MEX              ASSA ABLOY AB Class B  Mexican Stock Exchange   
2  1093N-MEX  CSPC Pharmaceutical Group Limited  Mexican Stock Exchange   
3  1113N-MEX          CK Asset Holdings Limited  Mexican Stock Exchange   
4  1299N-MEX                  AIA Group Limited  Mexican Stock Exchange   

         DATE  PRICE_DATE CURRENCY    PRICE  PRICE_OPEN  PRICE_HIGH  \
0  2024-03-31  2024-03-20      MXN  2082.59         NaN         NaN   
1  2024-03-31  2024-03-11      MXN   424.56         NaN         NaN   
2  2024-03-31  2024-03-26      MXN    15.69         NaN         NaN   
3  2024-03-31  2024-03-27      MXN    68.43         NaN         NaN   
4  2024-03-31  2024-03-22      MXN   125.87         NaN         NaN   

   PRICE_LOW  VOLUME  
0        NaN   627.0  
1        NaN  3179.0  
2        NaN     0.0  
3   

In [11]:

# Filter the DataFrame for rows where the ticker is 'CVX' and 'MDLZ'
cvx_data = price_1Y[price_1Y['TICKER'] == 'CVX-US'][['TICKER', 'PRICE_DATE', 'PRICE']]
mdlz_data = price_1Y[price_1Y['TICKER'] == 'MDLZ-US'][['TICKER', 'PRICE_DATE', 'PRICE']]
aapl_data = price_1Y[price_1Y['TICKER'] == 'AAPL-US'][['TICKER', 'PRICE_DATE', 'PRICE']]

# Merge the three datasets on PRICE_DATE to align them
merged_data_all = pd.merge(cvx_data, mdlz_data, on='PRICE_DATE', suffixes=('_CVX', '_MDLZ'))
merged_data_all = pd.merge(merged_data_all, aapl_data, on='PRICE_DATE')

# Rename the PRICE column for AAPL to avoid confusion
merged_data_all.rename(columns={'PRICE': 'PRICE_AAPL'}, inplace=True)

# Select only the PRICE columns for correlation calculation
price_data = merged_data_all[['PRICE_CVX', 'PRICE_MDLZ', 'PRICE_AAPL']]

print(merged_data_all.head())  # Display the first few rows of the merged DataFrame
# Calculate the pairwise correlation matrix
correlation_matrix = price_data.corr()

# Display the correlation matrix
print("Pairwise Correlation Matrix:")
print(correlation_matrix)

  TICKER_CVX  PRICE_DATE  PRICE_CVX TICKER_MDLZ  PRICE_MDLZ   TICKER  \
0     CVX-US  2024-03-28     157.74     MDLZ-US       70.00  AAPL-US   
1     CVX-US  2024-04-30     161.27     MDLZ-US       71.94  AAPL-US   
2     CVX-US  2024-05-31     162.30     MDLZ-US       68.53  AAPL-US   
3     CVX-US  2024-06-28     156.42     MDLZ-US       65.44  AAPL-US   
4     CVX-US  2024-07-31     160.47     MDLZ-US       68.35  AAPL-US   

   PRICE_AAPL  
0      171.48  
1      170.33  
2      192.25  
3      210.62  
4      222.08  
Pairwise Correlation Matrix:
            PRICE_CVX  PRICE_MDLZ  PRICE_AAPL
PRICE_CVX    1.000000    0.180972   -0.544940
PRICE_MDLZ   0.180972    1.000000   -0.518584
PRICE_AAPL  -0.544940   -0.518584    1.000000


In [28]:
#%pip install ib_insync
from ib_insync import *

# Connect to TWS or IB Gateway
ib = IB()

# Use asyncio to handle the event loop in Jupyter
async def main():
    # Connect to IBKR Gateway
    await ib.connectAsync('127.0.0.1', 4001, clientId=6)  # Use the correct port and clientId

    # Retrieve account information asynchronously
    account_summary = await ib.accountSummaryAsync()
    print("Account Summary:")
    for item in account_summary:
        print(f"{item.tag}: {item.value}")

    # Retrieve trading positions
    positions = ib.positions()
    print("\nTrading Positions:")
    for position in positions:
        print(f"Contract: {position.contract.symbol}, Position: {position.position}, Avg Cost: {position.avgCost}")

    # Disconnect from IBKR Gateway
    ib.disconnect()

# Run the main function in Jupyter
await main()

Account Summary:
AccountType: INDIVIDUAL
Cushion: 0.657678
LookAheadNextChange: 1743235200
AccruedCash: -19.36
AvailableFunds: 15915.21
BuyingPower: 106101.43
EquityWithLoanValue: 26234.56
ExcessLiquidity: 17254.26
FullAvailableFunds: 15915.21
FullExcessLiquidity: 17254.26
FullInitMarginReq: 10319.35
FullMaintMarginReq: 8980.87
GrossPositionValue: 36388.23
InitMarginReq: 10319.35
LookAheadAvailableFunds: 15915.21
LookAheadExcessLiquidity: 17254.26
LookAheadInitMarginReq: 10319.35
LookAheadMaintMarginReq: 8980.87
MaintMarginReq: 8980.87
NetLiquidation: 26235.13
TotalCashValue: -10133.74
AccountType: INDIVIDUAL
Cushion: 1
LookAheadNextChange: 1743235200
AccruedCash: 0.00
AvailableFunds: 4550.51
BuyingPower: 30336.71
EquityWithLoanValue: 5969.24
ExcessLiquidity: 4735.56
FullAvailableFunds: 4550.51
FullExcessLiquidity: 4735.56
FullInitMarginReq: 1418.73
FullMaintMarginReq: 1233.68
GrossPositionValue: 4934.72
InitMarginReq: 1418.73
LookAheadAvailableFunds: 4550.51
LookAheadExcessLiquidity: 

In [30]:
import requests

url = f'https://eodhd.com/api/real-time/CVX.US?api_token=66c0aeb1357b15.87356825&fmt=json'
data = requests.get(url).json()

print(data)

{'code': 'CVX.US', 'timestamp': 1743192780, 'gmtoffset': 0, 'open': 166.7, 'high': 167.13, 'low': 165.295, 'close': 166.09, 'volume': 4840078, 'previousClose': 166.65, 'change': -0.56, 'change_p': -0.336}


In [38]:
#%pip install ib_insync
from ib_insync import *

# Connect to TWS or IB Gateway
ib = IB()

# Use asyncio to handle the event loop in Jupyter
async def main():
    # Connect to IBKR Gateway
    await ib.connectAsync('127.0.0.1', 4001, clientId=9)  # Use the correct port and clientId

    # Define the contract for IUSA (e.g., iShares Core MSCI Europe ETF)
    contract = Stock('IUSA', 'SMART', 'EUR')  # Replace with the correct exchange and currency if needed

    # Qualify the contract asynchronously to ensure it is valid
    await ib.qualifyContractsAsync(contract)

    # Specify your account (replace 'U1234567' with your actual account ID)
    account = 'U11507596'  # Replace with your IBKR account ID

    # Define the limit order with the account specified
    limit_order = LimitOrder('BUY', 40, 40.00, account=account)  # Buy 40 shares at a limit price of 40 EUR

    # Place the order
    trade = ib.placeOrder(contract, limit_order)

      

    # Print the order status
    print(f"Order Status: {trade.orderStatus.status}")
    print(f"Filled: {trade.orderStatus.filled}, Remaining: {trade.orderStatus.remaining}")

    # Disconnect from IBKR Gateway
    ib.disconnect()

# Run the main function in Jupyter
await main()

Order Status: PendingSubmit
Filled: 0.0, Remaining: 0.0


In [47]:
#%pip install ib_insync
from ib_insync import *

# Connect to TWS or IB Gateway
ib = IB()

# Use asyncio to handle the event loop in Jupyter
async def main():
    # Connect to IBKR Gateway
    await ib.connectAsync('127.0.0.1', 4001, clientId=11)  # Use the correct port and clientId

    # Retrieve all open orders asynchronously
    open_orders = await ib.reqOpenOrdersAsync()
    print(f"Found {len(open_orders)} open orders.")

    # Cancel each open order
    for trade in open_orders:
        print(f"Cancelling order: {trade.order}")
        ib.cancelOrder(trade.order)

    # Wait for cancellation to complete
    await asyncio.sleep(2)  # Allow time for cancellations to process

    # Verify that all orders are canceled
    open_orders_after = await ib.reqOpenOrdersAsync()
    print(f"Remaining open orders after cancellation: {len(open_orders_after)}")

    # Disconnect from IBKR Gateway
    ib.disconnect()

# Run the main function in Jupyter
await main()

Found 0 open orders.
Remaining open orders after cancellation: 0


In [53]:
#%pip install ib_insync
from ib_insync import *

# Generate a list of 10 client IDs (e.g., 1 to 10)
client_ids = [i for i in range(1, 12)]  # Generates client IDs from 1 to 10

# Function to cancel all open orders and disconnect for a specific client ID
async def cancel_orders_and_disconnect(client_id):
    print(f"Connecting to IBKR Gateway with client ID: {client_id}")
    ib = IB()
    try:
        # Attempt to connect to IBKR Gateway
        await ib.connectAsync('127.0.0.1', 4001, clientId=client_id)
        print(f"Connected with client ID: {client_id}")
    except Exception as e:
        print(f"Error connecting with client ID {client_id}: {e}")
    finally:
        # Disconnect from IBKR Gateway
        print(f"Disconnecting client ID: {client_id}")
        ib.disconnect()

# Main function to cancel orders and disconnect for all client IDs
async def main():
    for client_id in client_ids:
        await cancel_orders_and_disconnect(client_id)

# Run the main function in Jupyter
await main()

Connecting to IBKR Gateway with client ID: 1


Peer closed connection. clientId 1 already in use?
API connection failed: TimeoutError()


Error connecting with client ID 1: 
Disconnecting client ID: 1
Connecting to IBKR Gateway with client ID: 2
Connected with client ID: 2
Disconnecting client ID: 2
Connecting to IBKR Gateway with client ID: 3


Peer closed connection. clientId 3 already in use?
API connection failed: TimeoutError()


Error connecting with client ID 3: 
Disconnecting client ID: 3
Connecting to IBKR Gateway with client ID: 4


...

Error connecting with client ID 7: 
Disconnecting client ID: 7
Connecting to IBKR Gateway with client ID: 8


Peer closed connection. clientId 8 already in use?
API connection failed: TimeoutError()


Error connecting with client ID 8: 
Disconnecting client ID: 8
Connecting to IBKR Gateway with client ID: 9
Connected with client ID: 9
Disconnecting client ID: 9
Connecting to IBKR Gateway with client ID: 10


Peer closed connection. clientId 10 already in use?
API connection failed: TimeoutError()


Error connecting with client ID 10: 
Disconnecting client ID: 10
Connecting to IBKR Gateway with client ID: 11
Connected with client ID: 11
Disconnecting client ID: 11
